<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DVC-+-MLflow-=-experiments-managementm" data-toc-modified-id="DVC-+-MLflow-=-experiments-managementm-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>DVC + MLflow = experiments managementm</a></span><ul class="toc-item"><li><span><a href="#Create-a-new-log_metrics-pipeline-stage" data-toc-modified-id="Create-a-new-log_metrics-pipeline-stage-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Create a new <code>log_metrics</code> pipeline stage</a></span></li></ul></li><li><span><a href="#Experimenting" data-toc-modified-id="Experimenting-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Experimenting</a></span><ul class="toc-item"><li><span><a href="#Experiment-1" data-toc-modified-id="Experiment-1-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Experiment 1</a></span></li><li><span><a href="#Experiment-2:-params-tuning" data-toc-modified-id="Experiment-2:-params-tuning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Experiment 2: params tuning</a></span></li></ul></li><li><span><a href="#Show-metrics-and-params-diffs-with-DVC" data-toc-modified-id="Show-metrics-and-params-diffs-with-DVC-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Show metrics and params diffs with DVC</a></span></li></ul></div>

# DVC + MLflow = experiments managementm

## Create a new `log_metrics` pipeline stage

Then we can add it to DVC as a stage and run together with other stades through `dvc repro`

Install `catboost` flavour which is not supported out of the box: 

In [1]:
! pip install src/utils/cb_flavor/.

Processing ./src/utils/cb_flavor
Using legacy 'setup.py install' for cb-flavor, since package 'wheel' is not installed.
    Running setup.py install for cb-flavor ... done
You should consider upgrading via the '/Users/antongusarov/ML_REPA/tutorial-predict-device-change/venv/bin/python -m pip install --upgrade pip' command.


Check that the newly created `log_metrics` pipeline stage is functioning properly:

In [1]:
! python -m src.pipelines.log_metrics --config=config/params.yaml

2021-02-25 23:18:53,090 — LOG_METRICS — INFO — Start logging
2021-02-25 23:18:53,100 — LOG_METRICS — DEBUG — Experiment ID: 3, Run ID: f5a3393ee58544a0aef78787a8e41fa9
2021-02-25 23:18:53,104 — LOG_METRICS — INFO — Log train metrics
2021-02-25 23:18:53,109 — LOG_METRICS — INFO — Log cross-validation metrics (raw metrics, by folds
2021-02-25 23:18:53,112 — LOG_METRICS — DEBUG — lift: {'2020-05-31': 2.1466859497062547, '2020-06-30': 2.1505471897179307, '2020-07-31': 2.1445188225607903, '2020-08-31': 2.1383669439425}
2021-02-25 23:18:53,117 — LOG_METRICS — DEBUG — precision_at_k: {'2020-05-31': 0.8380311652395894, '2020-06-30': 0.8377120672233155, '2020-07-31': 0.8346540445673563, '2020-08-31': 0.8354251981066851}
2021-02-25 23:18:53,124 — LOG_METRICS — DEBUG — recall_at_k: {'2020-05-31': 1.0, '2020-06-30': 1.0, '2020-07-31': 1.0, '2020-08-31': 1.0}
2021-02-25 23:18:53,132 — LOG_METRICS — INFO — Log params
2021-02-25 23:18:53,136 — LOG_METRICS — INFO — Log artifacts
2021-02-25 23:18:53,61

! mlflow ui

Add this new pipeline stage into DVC (`dvc.yaml`):

In [11]:
%%bash
dvc run -f \
    -n log_metrics \
    -d data/processed/features.feather \
    -d models/model.joblib \
    -d reports/train_metrics.json \
    -d reports/raw_metrics.csv \
    -O reports/mlflow_report.md\
    -p config/params.yaml:train,log_metrics \
    python -m src.pipelines.log_metrics --config=config/params.yaml

Running stage 'log_metrics' with command:
	python -m src.pipelines.log_metrics --config=config/params.yaml
2021-02-26 00:11:36,720 — LOG_METRICS — INFO — Start logging
2021-02-26 00:11:36,735 — LOG_METRICS — DEBUG — Experiment ID: 3, Run ID: 7100759601764c9ea53d15fa91a3b60b
2021-02-26 00:11:36,739 — LOG_METRICS — INFO — Log train metrics
2021-02-26 00:11:36,744 — LOG_METRICS — INFO — Log cross-validation metrics (raw metrics, by folds
2021-02-26 00:11:36,751 — LOG_METRICS — DEBUG — lift: {'2020-05-31': 2.038653459900317, '2020-06-30': 2.0630316674750206, '2020-07-31': 2.04804690407424, '2020-08-31': 2.054648213922152}
2021-02-26 00:11:36,760 — LOG_METRICS — DEBUG — precision_at_k: {'2020-05-31': 0.7958570440887093, '2020-06-30': 0.8036217624847098, '2020-07-31': 0.7971068446524491, '2020-08-31': 0.8027176514386002}
2021-02-26 00:11:36,766 — LOG_METRICS — DEBUG — recall_at_k: {'2020-05-31': 1.0, '2020-06-30': 1.0, '2020-07-31': 1.0, '2020-08-31': 1.0}
2021-02-26 00:11:36,772 — LOG_METRI

In [12]:
! dvc dag

7=






















+-----------------+  
| load_data_stage |  
+-----------------+  
+-----------------+  
| featurize_stage |  
+-----------------+  
+--------------------+               +-------------+  
| data/processed.dvc |               | train_stage |  
+--------------------+               +-------------+  
                   **               **                
                     ***         ***                  
                        **     **                     
                    +-------------+                   
                    | log_metrics |                   
                    +-------------+                   
~
~
~
~
~
~
~
~
(END)p/zg2nfhts6q74ptj2dn93gxfr0000gn/T/tmpnelsbodb (END)

# Experimenting

## Experiment 1

- Create a new branch `experiment_1`

In [22]:
! git checkout -b "experiment_1"

fatal: A branch named 'experiment_1' already exists.


- Modify any train parameters in `params.yaml`

For example:

```yaml
train:
  estimator: catboost
  catboost_params:
    iterations: 10
    thread_count: 50
    has_time: true
    depth: 6
    learning_rate: 0.15
    allow_writing_files: false
```

- Run experiment:

In [23]:
! dvc repro

Stage 'load_data_stage' didn't change, skipping                       core>
'data/processed.dvc' didn't change, skipping
Stage 'train_stage' didn't change, skipping
Stage 'log_metrics' didn't change, skipping
Stage 'featurize_stage' didn't change, skipping
Data and pipelines are up to date.


Note that `dvc.lock` has changed accordingly reflecting the updated model paramaters.

- Commit the experiment:

```bash
git add .
git commit -m "Make experiment 1"
```

- **Add tag**
Use MLflow `run_id` as a part of the tag 
```bash
git tag exp-1-[MLFLOW_RUN_ID] -a
```

```bash 
! git tag exp-1-89997d6ead9845babd4d6263b85bc217 -a
```

In [24]:
! git tag

exp-1-89997d6ead9845babd4d6263b85bc217
exp_1.0
v0.1
v0.2


- **DVC Push** using `dvc push`:

In [26]:
! dvc push -r myremote

Everything is up to date.                                                       


- **Push to `origin experiments`**:

In [27]:
! git push origin experiment_1

Enumerating objects: 19, done.
Counting objects: 100% (19/19), done.
Delta compression using up to 8 threads
Compressing objects: 100% (10/10), done.
Writing objects: 100% (10/10), 29.94 KiB | 5.99 MiB/s, done.
Total 10 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
remote: This repository moved. Please use the new location:
remote:   git@github.com:AntonGusarov/MLOps-demo-project.git
remote: 
remote: Create a pull request for 'experiment_1' on GitHub by visiting:
remote:      https://github.com/AntonGusarov/MLOps-demo-project/pull/new/experiment_1
remote: 
To github.com:AntonGusarov/predict-device-change.git
 * [new branch]      experiment_1 -> experiment_1


**Create a Merge Request into & merge `dev`**

## Experiment 2: params tuning

**GitHub**:

- Create an issue: `Add experiment 2`
- Create a new branch from `exeriments`

**Git fetch** the experiment branch to local workspace

```bash
git fetch
git checkout <EXPERIMENT_BRANCH_NAME>
```

In [28]:
! git branch

  dvc_pipelines
* experiment_1
  master


In [29]:
! git fetch

From github.com:AntonGusarov/predict-device-change
 * [new branch]      experiment_2 -> origin/experiment_2


In [31]:
! git branch -a

  dvc_pipelines
* experiment_1
  master
  remotes/AntonGusarov/dvc_pipelines
  remotes/AntonGusarov/experiment_1
  remotes/AntonGusarov/master
  remotes/origin/HEAD -> origin/master
  remotes/origin/dvc_pipelines
  remotes/origin/experiment_1
  remotes/origin/experiment_2
  remotes/origin/master


In [32]:
! git checkout remotes/origin/experiment_2

D	1.1-antong_step_3_unittests.ipynb
D	2.1-antong_step4_dvc.ipynb
A	Untitled1.ipynb
A	step-4.1-metrics-tracking.ipynb
A	step-4.2-model-life-cycle-management.ipynb
A	step-4.3-experiment-management.ipynb
Note: checking out 'remotes/origin/experiment_2'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 92d2082 Merge pull request #3 from AntonGusarov/experiment_1


Used PyCharm to checkout from a remote branch (automatically creates a local copy of a remote branch). 

In [34]:
! git branch -a

  dvc_pipelines
  experiment_1
* experiment_2
  master
  remotes/AntonGusarov/dvc_pipelines
  remotes/AntonGusarov/experiment_1
  remotes/AntonGusarov/master
  remotes/origin/HEAD -> origin/master
  remotes/origin/dvc_pipelines
  remotes/origin/experiment_1
  remotes/origin/experiment_2
  remotes/origin/master


**Modify any train parameters in `params.yaml`**

For example:

```yaml
base:
  ...
  exp_name: 'Experiment 2'
  
catboost_params:
    iterations: 30
```

- **Run experiment** via DVC pipeline

In [35]:
! dvc repro

'data/processed.dvc' didn't change, skipping                          core>
Running stage 'train_stage' with command:
	python -m src.pipelines.train --config=config/params.yaml
{'project_dir': '.', 'random_state': 42, 'log_level': 'DEBUG', 'exp_name': 'Experiment 1'}
2021-02-26 13:04:57,784 — TRAIN — INFO — Load data
2021-02-26 13:04:58,358 — TRAIN — INFO — Instantiate model
2021-02-26 13:04:58,445 — TRAIN — INFO — Top_K 5.0% of the dataset size: 37606
2021-02-26 13:04:58,446 — TRAIN — INFO — Fold 1:
2021-02-26 13:04:58,446 — TRAIN — INFO — Train: 2020-04-30 00:00:00 - 2020-04-30 00:00:00
2021-02-26 13:04:58,446 — TRAIN — INFO — Test: 2020-05-31 00:00:00 

2021-02-26 13:04:58,566 — TRAIN — INFO — Train shapes: X - (150484, 30), y - (150484,)
2021-02-26 13:04:58,567 — TRAIN — INFO — Test shapes: X - (150411, 30), y - (150411,)
0:	learn: 0.6627438	total: 192ms	remaining: 5.58s
1:	learn: 0.6299553	total: 297ms	remaining: 4.15s
2:	learn: 0.6057897	total: 409ms	remaining: 3.68s
3:	learn: 0.

13:	learn: 0.5017730	total: 2.44s	remaining: 2.79s
14:	learn: 0.4969726	total: 2.61s	remaining: 2.61s
15:	learn: 0.4941473	total: 2.79s	remaining: 2.44s
16:	learn: 0.4919248	total: 2.97s	remaining: 2.27s
17:	learn: 0.4895653	total: 3.14s	remaining: 2.09s
18:	learn: 0.4854192	total: 3.31s	remaining: 1.91s
19:	learn: 0.4829664	total: 3.5s	remaining: 1.75s
20:	learn: 0.4811894	total: 3.67s	remaining: 1.57s
21:	learn: 0.4781727	total: 3.85s	remaining: 1.4s
22:	learn: 0.4760381	total: 4.03s	remaining: 1.23s
23:	learn: 0.4745870	total: 4.2s	remaining: 1.05s
24:	learn: 0.4720071	total: 4.37s	remaining: 874ms
25:	learn: 0.4692665	total: 4.57s	remaining: 703ms
26:	learn: 0.4667085	total: 4.78s	remaining: 531ms
27:	learn: 0.4641235	total: 4.96s	remaining: 354ms
28:	learn: 0.4628421	total: 5.14s	remaining: 177ms
29:	learn: 0.4609297	total: 5.32s	remaining: 0us
2021-02-26 13:05:19,277 — TRAIN — INFO — Max probas: 0.9436699116130296
2021-02-26 13:05:19,318 — src.evaluate.metrics — INFO — Lift: 0.83

We can now check the results in MLflow server and `mlflow_report.md` to check the current experiment and run details. 

- **Commit experiment**

- **Add tag** using PyCharm

In [37]:
! git tag

exp-1-89997d6ead9845babd4d6263b85bc217
exp-2-dca173dc365e4ab5b4bb7decc0e83bab
exp_1.0
v0.1
v0.2


- **git push**

In [41]:
! git branch

  dvc_pipelines
  experiment_1
* experiment_2
  master


In [42]:
! git push

Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 8 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 848 bytes | 424.00 KiB/s, done.
Total 7 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
remote: This repository moved. Please use the new location:
remote:   git@github.com:AntonGusarov/MLOps-demo-project.git
To github.com:AntonGusarov/predict-device-change.git
   92d2082..dfd70aa  experiment_2 -> experiment_2


# Show metrics and params diffs with DVC

In [43]:
! dvc metrics show

	reports/train_metrics.json:                                          core>
		lift_max: 2.1490453632831845
		lift_min: 2.1325815032500364
		lift_std: 0.007060729528481228
		lift_mean: 2.141945072368462
		precision_at_k_max: 0.8374461522097537
		precision_at_k_min: 0.833164920491411
		precision_at_k_std: 0.0023529597322963485
		precision_at_k_mean: 0.8352523533478701
		recall_at_k_max: 1.0
		recall_at_k_min: 1.0
		recall_at_k_std: 0.0
		recall_at_k_mean: 1.0


In [45]:
!dvc metrics diff exp-1-89997d6ead9845babd4d6263b85bc217 exp-2-dca173dc365e4ab5b4bb7decc0e83bab

Path                        Metric               Old      New      Changee>
reports/train_metrics.json  lift_max             2.0829   2.14905  0.06615
reports/train_metrics.json  lift_mean            2.07286  2.14195  0.06908
reports/train_metrics.json  lift_min             2.06383  2.13258  0.06875
reports/train_metrics.json  lift_std             0.00785  0.00706  -0.00079
reports/train_metrics.json  precision_at_k_max   0.81136  0.83745  0.02609
reports/train_metrics.json  precision_at_k_mean  0.80832  0.83525  0.02694
reports/train_metrics.json  precision_at_k_min   0.80325  0.83316  0.02992
reports/train_metrics.json  precision_at_k_std   0.00356  0.00235  -0.0012
